In [94]:
# Data Processing
import pandas as pd
import numpy as np

# Data Visualization
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

# Purpose 
**Here's what wine you should be drinking, according to how you take your coffee.**

# Data crawling 
- refers to www.wine21.com
- price : under ₩ 200,000
- use only items being sold in off-line stores like E-mart, Lotte, Costco, and etc.
- add 'aroma' column later

In [77]:
df_1 = pd.read_csv('./data/wine21.csv')
df_1

,title,image_link,winery,country,province,grape_type,wine_type,occasion,alcohol,sweetness,acidity,body_rate,tannin_rate,food_pairing,price,vintage,size
0,트리풀라 피에몬테 로쏘,http://img.wine21.com/WINE_MST/TITLE/0167000/W...,몬도델비노,이탈리아,피에몬테,"['바르베라', '네비올로']",레드,테이블 와인,13.5,1,3,3,3,"피자, 파스타, 붉은 육류, 그리고 치즈 플레이트와 잘 어울리며, 특유의 산도가...",68000,2017,750
1,아케시 로사토 브뤼,http://img.wine21.com/WINE_MST/TITLE/0167000/W...,몬도델비노,이탈리아,피에몬테,"['바르베라', '돌체토']",스파클링,"에피타이저, 테이블 와인",11.5,1,3,3,1,식전 와인으로 적합하며 전체요리나 기름진 생선요리와 잘 어울린다.,55000,NV,750
2,엘리자베스 로제 샤도네이,http://img.wine21.com/WINE_MST/TITLE/0167000/W...,고스트 블락 와이너리,미국,캘리포니아,['샤르도네'],화이트,"에피타이저, 테이블 와인",13.5,1,4,3,1,"치즈, 해산물, 샐러드 등과 잘 어울린다.",135000,2018,750
3,"맥매니스, 캘리포니아 피노 누아",http://img.wine21.com/WINE_MST/TITLE/0167000/W...,맥매니스 와이너리,미국,캘리포니아,['피노 누아'],레드,테이블 와인,13.5,1,4,3,3,"치즈, 오리고기 등과 잘 어울린다.",65000,2019,750
4,"백하우스, 피노 누아",http://img.wine21.com/WINE_MST/TITLE/0167000/W...,백하우스,미국,캘리포니아,['피노 누아'],레드,테이블 와인,13.5,1,4,3,3,"치즈, BBQ, 구운 고기, 버섯 요리 등과 잘 어울린다.",48000,2018,750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5706,지네스테 메독,http://img.wine21.com/WINE_MST/TITLE/0137000/W...,메종 지네스테,프랑스,보르도,"['카베르네 소비뇽', '메를로']",레드,테이블 와인,8.5,1,3,3,3,붉은 육류와 강한 치즈등과 잘 어울린다.,34000,2007,750
5707,마스까롱 보르도 루즈,http://img.wine21.com/WINE_MST/TITLE/0137000/W...,메종 지네스테,프랑스,보르도,"['메를로', '카베르네 소비뇽']",레드,테이블 와인,13.0,1,3,3,3,"육류요리와 가금류, 치즈등과 잘 어울린다.",36000,2007,750
5708,마스까롱 보르도 화이트,http://img.wine21.com/WINE_MST/TITLE/0137000/W...,메종 지네스테,프랑스,보르도,"['세미용', '소비뇽 블랑']",화이트,테이블 와인,8.5,1,3,2,1,"구운 생선, 치즈 등과 잘 어울린다.",36000,2008,750
5709,지네스떼 보르도 화이트,http://img.wine21.com/WINE_MST/TITLE/0137000/W...,메종 지네스테,프랑스,보르도,"['세미용', '소비뇽 블랑']",화이트,테이블 와인,12.0,1,4,2,1,"생선요리와, 해산물등과 잘 어울린다.",24000,2010,750


# Data Processing

In [78]:
df = df_1.copy()

## grape_type : use the first species

In [79]:
for i in range(len(df)):
    if ',' in df['grape_type'][i]:
        df['grape_type'][i] = df['grape_type'][i][2:-2].split(',')[0][:-1]
    else:
        df['grape_type'][i] = df['grape_type'][i][2:-2]
df['grape_type']

0           바르베라
1           바르베라
2           샤르도네
3          피노 누아
4          피노 누아
          ...   
5706    카베르네 소비뇽
5707         메를로
5708         세미용
5709         세미용
5710         메를로
Name: grape_type, Length: 5711, dtype: object

In [75]:
df['grape_type'].unique()

array(['바르베라', '샤르도네', '피노 누아', '카베르네 소비뇽', '메를로', '마르산느', '산지오베제',
       '비오시뉴', '투리가 프란체사', '시라/쉬라즈', '말벡', '슈냉 블랑', '진판델', '모나스트렐',
       '소비뇽 블랑', '그르나슈', '쌩쏘', '모스카토', '블랜드', '템프라니요', '네로 다볼라', '프리미티보',
       '네비올로', '카르메네르', '투리가 나시오날', '자렐로', '베르데호', '아시르티코', '시노마브로',
       '마브루드', '아기오르기티코', '로디티스', '빌라나', '까리냥', '모스카텔 세투발', '가르나차', '피아노',
       '마카베오', '틴타 네그라 몰레', '아이렌', '리슬링', '글레라', '오미자', '코르비나 베로네제',
       '쁘띠 시라', '누라구스', '소비뇽 그리', '베르디키오 비앙코', '몬테풀치아노', '라크리마', '비오니에',
       '모스카텔', '푸르민트', '젤렌', '트라하두라', '그르나슈 블랑', '멘시아', '보발', '돌체토',
       '코르테제', '아르네이스', '뮈스까 까넬리', '세르시알', '알리아니코', '피노 네로', '세미용',
       '베르멘티노', '카베르네 프랑', '믈롱 드 부르고뉴', '피노 그리지오', '트라미너', '비달', '피노 뮈니에',
       '가르나차 틴토레라', '레불라', '쁘띠 메슬리에', '모작', '네로 디 트로이아', '레포스코', '투르비아노',
       '말바시아 디 칸디다', '클레렛', '피노 그리', '페르나오 피레스', '아라고네즈', '코르비나', '론디넬라',
       '롤', '카스텔라옹', '비칼', '사그란티노', '고데요', '트레비아노', '알바리뇨', '플라바치 말리',
       '포십', '게뷔르츠트라미너', '모스까뗄 데 알레한드리아', '오르테가', '틴타 로리즈', '청포도 즙',
    

## food_pairing : one_hot encoding
- add meat / cheese / seafood / vegetables / dessert columns
- meat : 고기, 육류, 돼지, 소, 비프,  닭, 오리, 양, 스테이크, 보쌈, 냉채, 바비큐, 하몽, 불고기, 햄, 쇠고기, 소시지, 칠면조, 치킨,
- cheese : 치즈
- seafood : 해산물, 조개, 새우, 갑각류, 연어, 참치, 초밥, 생선, 굴, 회, 사시미, 관자,
- vegetables : 버섯, 딸기, 올리브, 아몬드, 샐러드, 아보카도, 과일, 호두, 멜론, 무화과, 야채, 견과류, 
- dessert : 디저트, 수플레, 케익, 케이크, 파이, 마카롱, 아이스크림,

In [85]:
for i in range(len(df['food_pairing'])):
    if '디저트' in df['food_pairing'][i]:
        print(df['food_pairing'][i])

샐러드, 과일, 디저트 등과 잘 어울린다.

치즈, 육류 뿐만 아니라 초콜릿과 같은 달콤한 디저트 등과도 잘 어울린다.
브리 또는 가벼운 치즈, 건 과일, 캐러멜 또는 커피, 토피 디저트, 호두 파이, 티라미슈, 무화과 타르트 등과 잘 어울린다.
달콤한 디저트, 다양한 빵, 생크림을 올린 케이크 등과 잘 어울린다.
전채 음식, 디저트, 빠에야, 샐럳, 구운 고기 등과 잘 어울린다.
전채 요리, 디저트 등과 잘 어울린다.
생선, 조개 및 타파스, 디저트, 유기농 식품, 식전주 등으로 잘 어울린다.
청포도, 오렌지 등 달콤하면서도 신맛이 나는 과일 및 디저트 등과  잘 어울린다. 
그린 샐러드, 돼지고기, 디저트 등과 잘 어울린다.
식전주로도 좋으며 파테, 푸아그라, 모든 종류의 디저트류와 잘 어울린다.
소고기, 오리고기, 베리류의 디저트와 잘 어울린다.
샐러드, 신선한 과일, 가벼운 디저트 등과 잘 어울린다.
에피타이저, 디저트, 해산물, 과일류 등과 잘 어울린다.
에피타이저, 초콜렛, 디저트, 해산물 등과 잘 어울린다.
아페리티브, 각종 디저트, 에끌레어, 해산물 요리 등과 잘 어울린다.
샐러드, 콜드 컷, 닭고기 요리, 과일이 들어간 디저트류 등과 잘 어울린다.
달콤한 디저트, 파이, 짭조름한 음식, 칵테일 베이스 등으로 잘 어울린다.
관자 요리, 푸아그라, 디저트, 샐러드 등과 잘 어울린다.
해산물, 파스타, 치즈, 샐러드, 디저트류 등과 잘 어울린다.
해산물, 파스타, 치즈, 샐러드, 디저트류 등과 잘 어울린다.
해산물, 파스타, 매운 음식, 샐러드, 디저트류 등과 잘 어울린다.
해산물, 파스타, 매운 음식, 샐러드, 디저트류 등과 잘 어울린다.
페이스트리, 타르트 등의 디저트 류와 잘 어울린다.
과일, 치즈, 케이크 등의 디저트류 및 아페리티프 등과 잘 어울린다.
식전주, 디저트, 케이크, 아이스크림, 과일 샐러드 등과 잘 어울린다.
생선, 디저트, 스위트 식품 등과 잘 어울린다.
과일 케이크, 디저트류 등과 잘 어울린다.
식전 에피타이저, 해산물, 디

## continent
- Europe_A : 프랑스, 오스트리아, 몰도바, 독일, 헝가리, 슬로베니아, 불가리아, 조지아, 루마니아
- Europe_B : 이탈리아, 포르투갈, 스페인, 그리스, 크로아티아, 이스라엘, 레바논
- North_America : 미국, 캐나다
- South_America : 아르헨티나, 칠레, 우루과이
- New_World : 호주, 뉴질랜드, 남아프리카
- Korea : 대한민국

In [86]:
df['country'].unique()

array(['이탈리아', '미국', '프랑스', '포르투갈', '아르헨티나', '칠레', '스페인', '호주', '그리스',
       '오스트리아', '뉴질랜드', '몰도바', '독일', '대한민국', '헝가리', '슬로베니아', '남아프리카',
       '캐나다', '불가리아', '크로아티아', '우루과이', '조지아', '루마니아', '이스라엘', '레바논'],
      dtype=object)

In [87]:
# df[(df['sweetness'] == 1) & (df['body_rate'] >= 3) & (df['tannin_rate'] >= 3) & (df['wine_type'] == '레드')]

## price range

# Customer's Review Pivot Table

###### Survey

> 1) How do you take your coffee?
1. I love black coffee
2. With milk & sweets (eg. Frappuccino, Vanila Latte and etc)
3. I'm a tea-drinker

> 2) Which type of bean do you prefer?
1. Acidic one
2. Full_bodied one

###### cases (20 cases per a type)

1. type A : black - acidic : Dry / light / high acidity / dark flavors
2. type B : black - bodied : Dry / bold / low acidity / dark flavors
3. type C : cafe latte with sweets - acidic : sweet / bold / high acidity 
4. type D : cafe latte with sweets - bodied : sweet / bold / low acidity 
5. type E : Tea - acidic : Dry / light / high acidity / fruity flavors 
6. type F : Tea - bodied : Dry / bold / low acidity / fruity flavors

In [63]:
# user based recommendation
# add customer's rating data

ratings_expand = {
    
    'taeheon': {
        'type' : 'A',
        'Aysen Reserva Merlot' : 4.0,
        'Song Blue Saint-Emillion': 3.0,
        'Song Blue Merlot': 3.0,
        "Francois Janoueix Ch' Trincaud": 3.5,
        "Santa Carolina Estrella Merlot": 4.0,
    },
    'dong': {
        'type' : 'A',
        'Joseph Drouhin La Foret Bourgogne Blanc' : 4.0,
        'Santa Carolina Estrella Merlot': 3.0,
        '[yellow tail] Merlot': 3.0,
        'L Merlot': 3.5,
        "Apothic Red Winemaker's Blend": 4.0,
    },
}

# User-based Recommendation
- recommend top-3 wines, considering cosign similarity of customer's type, price_range, food, aroma and continent

## split the data by customer's type

## Cosign Similarity

### price_range and food you have with

### preferred aroma

### continent 

# Kakao Plus Friends Chatbot